# Entitlements in TOC-eligible parcels

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import boto3
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

## Parcels
* Figure out how many are duplicates
* Won't know which AINs are used in PCTS, so keep all of them, but have a way to identify how many obs to drop later on

In [3]:
parcels = gpd.read_file(f'zip+s3://{bucket_name}/gis/intermediate/la_parcels_toc.zip')

display(parcels.TOC_Tier.value_counts())
parcels = parcels[parcels.TOC_Tier > 0]

0    442562
1    148994
3    110153
2     65653
4      7844
Name: TOC_Tier, dtype: int64

In [4]:
# Upload just the parcels in TOC Tiers into S3
parcels.to_file(driver = 'GeoJSON', filename = '../gis/intermediate/toc_eligible_parcels.geojson')

s3.upload_file('../gis/intermediate/toc_eligible_parcels.geojson', f'{bucket_name}', 'gis/intermediate/toc_eligible_parcels.geojson')

In [5]:
# Grab the centroids and count number of duplicate obs
parcels2 = utils.get_centroid(parcels)

## Grab tables from PCTS

In [6]:
cases = pd.read_parquet('../data/tCASE.parquet')
app = pd.read_parquet('../data/tAPLC.parquet')
geo_info = pd.read_parquet('../data/tPROP_GEO_INFO.parquet')
la_prop = pd.read_parquet('../data/tLA_PROP.parquet')
#cases = catalog.pcts.tCASE.read()
#app = catalog.pcts.tAPLC.read()
#geo_info = catalog.pcts.tPROP_GEO_INFO.read()
#la_prop = catalog.pcts.tLA_PROP.read()

In [7]:
cases1 = cases[['CASE_ID', 'APLC_ID', 'CASE_NBR', 'CASE_SEQ_NBR', 'CASE_YR_NBR', 
                'CASE_ACTION_ID', 'CASE_FILE_RCV_DT']]
app1 = app[['APLC_ID', 'PROJ_DESC_TXT']]
geo_info1 = geo_info[['CASE_ID', 'PROP_ID']]
la_prop1 = la_prop[la_prop.ASSR_PRCL_NBR.notna()][['PROP_ID', 'ASSR_PRCL_NBR']]

### Subset cases, keep Oct 2017 and after

In [8]:
# Grab the year-month from received date
cases1['CASE_FILE_DATE'] = pd.to_datetime(cases1['CASE_FILE_RCV_DT']).dt.to_period('M')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
cases2 = cases1[cases1.CASE_FILE_DATE >= '2017-10']

## Parent cases only

In [10]:
parents = pd.read_parquet('../data/parent_cases.parquet')

cases3 = pd.merge(cases2, parents, on = 'CASE_ID', how = 'inner', validate = '1:1')

## Merge with geo_info, la_prop, parcels to ID the parcels that and have entitlements (2016 - after)

In [11]:
m1 = pd.merge(cases3, geo_info1, on = 'CASE_ID', how = 'inner', validate = '1:m')

In [12]:
m2 = pd.merge(m1, la_prop1, on = 'PROP_ID', how = 'inner', validate = 'm:1')

In [13]:
m3 = pd.merge(m2, parcels, left_on = 'ASSR_PRCL_NBR', right_on = 'AIN', how = 'inner', validate = 'm:1')

## Join parcels to zoning and subset to eligible zones
* Subset by eligible zones, see how many TOC-eligible parcels also fall into eligible zones

In [14]:
zoning = gpd.read_file(f's3://{bucket_name}/gis/raw/parsed_zoning.geojson')

In [15]:
eligible_zones = ['R2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5', 
                  'RD1.5', 'RD2', 'RD3', 'RD4', 'RD5', 'RD6', 
                  'C1', 'C2', 'C4', 'C5']

eligible_zoning = zoning[zoning.zone_class.isin(eligible_zones)]

In [16]:
parcels_with_zoning = gpd.sjoin(parcels2, eligible_zoning, how = 'inner', op = 'intersects').drop(columns = ['index_right'])

In [17]:
# Merge in zoning and TOC info about the parcel
m4 = pd.merge(m3, parcels_with_zoning, on = ['AIN', 'centroid', 'TOC_Tier'], how = 'inner')

In [18]:
# Merge in project description
m5 = pd.merge(m4, app1, on = 'APLC_ID', how = 'inner', validate = 'm:1')

# Drop duplicates
m5 = m5.drop_duplicates()

## Parse the PCTS string and grab prefix
* Drop duplicates
* Drop ENV, ADM, PAR cases
* Subset by CASE_ACTION_ID

In [19]:
parsed_col_names = ['prefix']

def parse_pcts(row):
    try:
        z = utils.PCTSCaseNumber(row.CASE_NBR)
        return pd.Series([z.prefix], index = parsed_col_names)
    except ValueError:
        return pd.Series([z.prefix], index = parsed_col_names)

parsed = m5.apply(parse_pcts, axis = 1)

m6 = pd.concat([m5, parsed], axis = 1)

In [20]:
# Drop duplicates
drop = ['centroid', 'x', 'y', 'obs', 'num_obs', 
        'CASE_ID', 'APLC_ID', 'ASSR_PRCL_NBR', 'PROP_ID', 
        'o1', 'o2', 'o3', 'o1_descrip', 'o2_descrip', 'o3_descrip']

m6 = m6.drop(columns = drop)

# Create new id variable that is just seq number and year. Need a way to get rid of duplicate cases.
m6['id'] = m6.CASE_SEQ_NBR.astype(int).astype(str) + '_' + m6.CASE_YR_NBR.astype(int).astype(str)

In [21]:
cols_we_need = ['id', 'CASE_ACTION_ID', 'CASE_FILE_DATE', 'AIN', 'TOC_Tier', 'ZONE_CMPLT', 'PROJ_DESC_TXT']

m7 = m6.drop_duplicates(subset = cols_we_need)

In [22]:
# Subset by prefix
drop_prefix = ['ENV', 'ADM', 'PAR']

m7 = m7.loc[~m6.prefix.isin(drop_prefix)]

In [23]:
# Subset by CASE_ACTION_ID -- let's use all cases for now, 
display(m7.CASE_ACTION_ID.value_counts())

#approved_cases = [1, 2, 11]
approved_cases = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

m8 = m7.loc[m7.CASE_ACTION_ID.isin(approved_cases)]

2.0     1047
1.0      204
4.0       52
6.0       42
3.0       17
11.0      11
5.0        1
Name: CASE_ACTION_ID, dtype: int64

#### At this point, no more duplicates by PARENT_CASE-AIN

## Additional subsetting for TOC-eligible parcels

In [24]:
# Tag case as TOC or not
m8['is_TOC'] = m8.CASE_NBR.str.contains('TOC').astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
keep_col = ['CASE_NBR', 'id', 'CASE_ACTION_ID', 'CASE_FILE_DATE', 'AIN', 'TOC_Tier', 'zone_class', 'is_TOC']

m9 = m8[keep_col]

## Make into parcel level df

In [26]:
m10 = m9.groupby(['AIN', 'TOC_Tier', 'zone_class', 'is_TOC']).agg({'id':'count'}).reset_index()

In [27]:
# Make wide
m10['num_TOC'] = m10.apply(lambda row: row.id if row.is_TOC == 1 else np.nan, axis = 1) 
m10['num_nonTOC'] = m10.apply(lambda row: row.id if row.is_TOC == 0 else np.nan, axis = 1)

In [28]:
# If there are multiple obs for the same AIN, fill the NaNs with the max from the other column, so we can just drop duplicates after
for col in ['num_TOC', 'num_nonTOC']:
    m10[col] = m10[col].fillna(m10.groupby('AIN')[col].transform('max'))

In [29]:
m11 = m10.drop_duplicates(subset = ['AIN', 'TOC_Tier', 'zone_class', 'num_TOC', 'num_nonTOC'])

for col in ['num_TOC', 'num_nonTOC']:
    m11[col] = m11[col].fillna(0).astype(int)

m11 = m11.drop(columns = ['is_TOC', 'id'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
toc_parcels = m11[m11.num_TOC > 0]
non_toc_parcels = m11[m11.num_nonTOC > 0]
have_both_parcels = m11[(m11.num_TOC > 0) & (m11.num_nonTOC > 0)]

print(f'# parcels: {len(m11)}')
print(f'# parcels with TOC entitlements: {len(toc_parcels)}')
print(f'# parcels with non TOC entitlements: {len(non_toc_parcels)}')
print(f'# parcels with both TOC and non TOC entitlements: {len(have_both_parcels)}')
print(f'double check sum: {len(toc_parcels) + len(non_toc_parcels) - len(have_both_parcels)}')

# parcels: 1219
# parcels with TOC entitlements: 234
# parcels with non TOC entitlements: 1004
# parcels with both TOC and non TOC entitlements: 19
double check sum: 1219


In [31]:
print(f'% parcels with TOC entitlements: {len(toc_parcels) / len(m11)}')
print(f'% parcels with non TOC entitlements: {len(non_toc_parcels) / len(m11)}')
print(f'% parcels with both entitlements: {len(have_both_parcels) / len(m11)}')

% parcels with TOC entitlements: 0.19196062346185397
% parcels with non TOC entitlements: 0.8236259228876128
% parcels with both entitlements: 0.015586546349466776


In [32]:
m11.TOC_Tier.value_counts()

3    591
1    280
2    249
4     99
Name: TOC_Tier, dtype: int64

In [33]:
toc_parcels.zone_class.value_counts()

C2       86
R3       73
R4       50
C4       13
RD1.5     6
R5        2
RAS4      2
R2        1
RD2       1
Name: zone_class, dtype: int64

In [34]:
non_toc_parcels.zone_class.value_counts()

C2       420
C4       166
R3       107
RD1.5     91
R2        74
RD2       60
R4        30
C5        17
C1        13
RD3       10
R5         9
RAS4       6
RD5        1
Name: zone_class, dtype: int64

In [35]:
m12 = pd.merge(m11, parcels2, on = ['AIN', 'TOC_Tier'], how = 'inner').drop(columns = ['x', 'y', 'obs', 'num_obs'])

In [36]:
m12.rename(columns = {'centroid':'geometry'}, inplace = True)
m12 = gpd.GeoDataFrame(m12)
m12.crs = {'init':'epsg:2229'}

In [37]:
m12.to_file(driver = 'GeoJSON', filename = '../gis/intermediate/toc_eligible_parcels_with_entitlements.geojson')

s3.upload_file('../gis/intermediate/toc_eligible_parcels_with_entitlements.geojson', f'{bucket_name}', 'gis/intermediate/toc_eligible_parcels_with_entitlements.geojson')

## Breakdown by TOC Tiers

In [38]:
m13 = m12.groupby('TOC_Tier').agg({'AIN':'count', 'num_TOC':'sum', 'num_nonTOC':'sum'}).reset_index()

for i in ['TOC', 'nonTOC']:
    new_col = f'pct_{i}'
    numerator = f'num_{i}'
    m13[new_col] = m13[numerator] / (m13.num_TOC + m13.num_nonTOC)
    
m13['all_AIN'] = m13.AIN.sum()
m13['pct_AIN'] = m13.AIN / m13.all_AIN

m13

,TOC_Tier,AIN,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC,all_AIN,pct_AIN
0,1,280,31,279,0.100000,0.900000,1219,0.229696
1,2,249,55,203,0.213178,0.786822,1219,0.204266
2,3,591,138,520,0.209726,0.790274,1219,0.484824
3,4,99,11,137,0.074324,0.925676,1219,0.081214


## Breakdown by Zone Class

In [39]:
m14 = m12.groupby('zone_class').agg({'AIN': 'count', 'num_TOC': 'sum', 'num_nonTOC': 'sum'}).reset_index()

for i in ['TOC', 'nonTOC']:
    new_col = f'pct_{i}'
    numerator = f'num_{i}'
    m14[new_col] = m14[numerator] / (m14.num_TOC + m14.num_nonTOC)
    
m14['all_AIN'] = m14.AIN.sum()
m14['pct_AIN'] = m14.AIN / m14.all_AIN

m14

,zone_class,AIN,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC,all_AIN,pct_AIN
0,C1,13,0,14,0.000000,1.000000,1219,0.010664
1,C2,495,87,481,0.153169,0.846831,1219,0.406071
2,C4,177,13,211,0.058036,0.941964,1219,0.145201
3,C5,17,0,20,0.000000,1.000000,1219,0.013946
4,R2,74,1,78,0.012658,0.987342,1219,0.060705
5,R3,177,73,114,0.390374,0.609626,1219,0.145201
6,R4,79,50,31,0.617284,0.382716,1219,0.064807
7,R5,11,2,10,0.166667,0.833333,1219,0.009024
8,RAS4,8,2,7,0.222222,0.777778,1219,0.006563
9,RD1.5,96,6,99,0.057143,0.942857,1219,0.078753


In [40]:
writer = pd.ExcelWriter('../outputs/toc_charts.xlsx', engine = 'xlsxwriter')

m13.to_excel(writer, sheet_name = 'entitlements_by_tier')
m14.to_excel(writer, sheet_name = 'entitlements_by_zone')

writer.save()